<a href="https://colab.research.google.com/github/OliviaHutchison/IMLOProject/blob/main/CNN_Model_Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports + Setup

In [1]:
#check for GPU access with PyTorch
import torch
torch.cuda.is_available()

False

In [2]:
#setup DAC
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
#Import PyTorch
import torch
from torch import nn

#Torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize

#Version check
print(torch.__version__)
print(torchvision.__version__)

2.2.1+cu121
0.17.1+cu121


# Get Dataset

In [4]:
transform = transforms.Compose([
    transforms.RandomRotation(70),
    transforms.RandomAutocontrast(0.5),
    transforms.RandomAdjustSharpness(4, 0.5),
    transforms.RandomPosterize(4, 0.5),
    transforms.GaussianBlur(3),
    transforms.RandomRotation(90),
    transforms.RandomEqualize(0.5),
    ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),
    Resize((256, 256))
])

In [5]:
#Training Version
train_data= datasets.Flowers102(
    root="data", #where to download to
    split='train', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

100%|██████████| 344862509/344862509 [00:27<00:00, 12600417.41it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 664417.99it/s]


100%|██████████| 14989/14989 [00:00<00:00, 13130414.09it/s]


In [6]:
#Validation
val_data= datasets.Flowers102(
    root="data", #where to download to
    split='val', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

In [7]:
#Testing
test_data= datasets.Flowers102(
    root="data", #where to download to
    split='test', #training dataset
    download=True, #want to download
    transform=transform, #transform data
    target_transform=None #transform labels
)

# Prep Data

In [8]:
from torch.utils.data import DataLoader

In [9]:
batchSize = 32 #groups of 32 images

#Turn data into iterables
trainDataloader = DataLoader(dataset=train_data,
                             batch_size=batchSize,
                             shuffle=True)

valDataloader = DataLoader(dataset=val_data,
                            batch_size=batchSize,
                            shuffle=False)

testDataloader = DataLoader(dataset=test_data,
                            batch_size=batchSize,
                            shuffle=False)

# CNN

In [10]:
#Create CNN
class FlowersModel(nn.Module):
  def __init__(self, inputShape: int, hiddenUnits: int, outputShape: int):
    super().__init__()
    #convolutional block 1 - feature extractor
    self.conv_block1 = nn.Sequential(
        #create conv layer
        nn.Conv2d(in_channels=inputShape, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1), #condv2d for 2d data
        #values we set ourselves in NNs are hyperparameters

        #ReLu Layer
        nn.ReLU(),

        #Batch Norm Layer
        nn.BatchNorm2d(hiddenUnits),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #convolutional block 2 - feature extractor
    self.conv_block2 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #Batch Norm Layer
        nn.BatchNorm2d(hiddenUnits),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),
    )
    #convolutional block 3 - feature extractor
    self.conv_block3 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #Batch Norm Layer
        nn.BatchNorm2d(hiddenUnits),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #1st Softplus
        nn.Softplus(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),


    )
    #convolutional block 4 - feature extractor
    self.conv_block4 = nn.Sequential(
        #conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),
        #ReLu Layer
        nn.ReLU(),

        #Batch Norm Layer
        nn.BatchNorm2d(hiddenUnits),

        #2nd conv layer
        nn.Conv2d(in_channels=hiddenUnits, out_channels=hiddenUnits,
                  kernel_size=3, stride=1, padding=1),

        #2 ReLu Layer
        nn.ReLU(),

        #MaxPool Layer
        nn.MaxPool2d(kernel_size=2),


    )
    #classifier layer
    self.classifierFinal = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hiddenUnits*64*64,
                  out_features=outputShape),
    )

  #forward
  def forward(self, x):
    x = self.conv_block1(x)
    #print(x.shape)
    x = self.conv_block2(x)
    #print(x.shape)
    x = self.classifierFinal(x)
    return x


In [11]:
model1 = FlowersModel(inputShape=3,
                      hiddenUnits=102,
                      outputShape=102).to(device)

## Loss and Optimisation

In [12]:
##Loss and Optimiser
lossCalc = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(params=model1.parameters(),
                             lr=0.00001)

## Save Model

# Test and Training

## Setup and functions

In [13]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

### Accuracy

In [14]:
#Accuaracy Calculator
def accuracyCalc(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

### Training

In [15]:
#Training Function
def trainFunction(model: torch.nn.Module,
               dataLoader: torch.utils.data.DataLoader,
               lossCalc: torch.nn.Module,
               optimiser: torch.optim.Optimizer,
               accuracyCalc,
               device: torch.device = device):
    trainLoss, trainAcc = 0, 0
    for batch, (X, y) in enumerate(dataLoader):
        #Send data to GPU
        X, y = X.to(device), y.to(device)

        #Forward pass
        y_pred = model(X)

        #Calculate loss
        loss = lossCalc(y_pred, y)
        trainLoss += loss
        trainAcc += accuracyCalc(y_true=y,
                                 y_pred=y_pred.argmax(dim=1))

        #Optimizer zero grad
        optimiser.zero_grad()

        #Backpropagation
        loss.backward()

        #Optimiser step
        optimiser.step()

    # Calculate loss and accuracy per epoch
    trainLoss /= len(dataLoader)
    trainAcc /= len(dataLoader)
    print(f"Train loss: {trainLoss:.5f} | Train accuracy: {trainAcc:.2f}%")

### Validation

In [16]:
#Validation Function
def valFunction(dataLoader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              lossCalc: torch.nn.Module,
              accuracyCalc,
              device: torch.device = device):
    valLoss, valAcc = 0, 0
    model.eval()
    #Turn on inference context manager
    with torch.inference_mode():
        for X, y in dataLoader:
            #Send data to GPU
            X, y = X.to(device), y.to(device)

            #Forward pass
            val_pred = model(X)

            #Calculate loss and accuracy
            valLoss += lossCalc(val_pred, y)
            valAcc += accuracyCalc(y_true=y,
                y_pred=val_pred.argmax(dim=1)
            )

        #Adjust and display
        valLoss /= len(dataLoader)
        valAcc /= len(dataLoader)
        print(f"Val loss: {valLoss:.5f} | Val accuracy: {valAcc:.2f}%\n")

### Test

In [17]:
#Testing Function
def testFunction(dataLoader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              lossCalc: torch.nn.Module,
              accuracyCalc,
              device: torch.device = device):
    testLoss, testAcc = 0, 0
    model.eval()
    #Turn on inference context manager
    with torch.inference_mode():
        for X, y in dataLoader:
            #Send data to GPU
            X, y = X.to(device), y.to(device)

            #Forward pass
            test_pred = model(X)

            #Calculate loss and accuracy
            testLoss += lossCalc(test_pred, y)
            testAcc += accuracyCalc(y_true=y,
                y_pred=test_pred.argmax(dim=1)
            )

        #Adjust and display
        testLoss /= len(dataLoader)
        testAcc /= len(dataLoader)
        print(f"Test loss: {testLoss:.5f} | Test accuracy: {testAcc:.2f}%\n")

## Train and Test

In [18]:
torch.manual_seed(42)

# Measure time
from timeit import default_timer as timer
trainTimeStartModel1 = timer()

# Train and test model
epochs = 100
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    trainFunction(dataLoader=trainDataloader,
        model=model1,
        lossCalc=lossCalc,
        optimiser=optimiser,
        accuracyCalc=accuracyCalc,
        device=device
    )
    valFunction(dataLoader=valDataloader,
        model=model1,
        lossCalc=lossCalc,
        accuracyCalc=accuracyCalc,
        device=device
    )


testFunction(dataLoader=testDataloader,
    model=model1,
    lossCalc=lossCalc,
    accuracyCalc=accuracyCalc,
    device=device
)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
---------


KeyboardInterrupt: 

## Save Model

1. `torch.save()` - saving and loading object, save in pythons's pickle format

2. `torch.load()` - load with pickle

3. `torch.nn.Module.load_state_dict()` - load models saved state dictionary

In [ ]:
torch.save(model1.state_dict(), './CNNModel.pth')